# Time Series - Exploratory Data Analysis

This notebook explores various time series data exploration techniques including visualization, decomposition, and anomaly detection. It also covers methods for handling high-frequency data with Fourier transforms and wavelet analysis. 

Throughout, we will be building a general EDA framework and tool that can be reused for various time series datasets.

Table of Contents:

0. [Data Loading and Preprocessing](#data-loading-and-preprocessing)
1. [The time index](#the-time-index)
2. [Measures of Distribution](#measures-of-distribution)
3. [Stationarity](#stationarity)
4. [Self-Correlation](#self-correlation) 
5. [Putting it All Together: An EDA Class](#putting-it-all-together-an-eda-class)

In [1]:
import pandas as pd
import plotly.graph_objects as go
import statsmodels
from plotly.subplots import make_subplots
from pathlib import Path

# Data Loading and Preprocessing

Before we get to anything else, we need to load some data. For this notebook, we will use some energy demand data from Kaggle, showing the electrical demand in several European in megawatt hours (MWh) at one hour intervals. Because it is at the top of the alphabet, we will look at Austria's demand data.

In [2]:
column = 'AT_load_actual_entsoe_transparency'
fp = Path('../data/forecasting/load-wind-and-solar-prices-in-hourly-resolution/time_series_60min_singleindex.csv')
df = pd.read_csv(fp)

# index manipulation - to datetime, set as index
df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'])
df = df.set_index('utc_timestamp')

# getting the Austria load column
df = df.filter([column])
df = df.rename(columns={column: 'load'})

# drop missing values
df = df.dropna()

We now have the data loaded into a Pandas DataFrame called `df` as a single column time series with a datetime index. Let's verify this by checking the first few rows of the DataFrame.

In [3]:
df.head()

,load
utc_timestamp,
2015-01-01 00:00:00+00:00,5946.0
2015-01-01 01:00:00+00:00,5726.0
2015-01-01 02:00:00+00:00,5347.0
2015-01-01 03:00:00+00:00,5249.0
2015-01-01 04:00:00+00:00,5309.0


# The time index

For any data series of temporally indexed data, making sense of the time index is crucial. Let's start by examining the time index of our DataFrame `df`. We want to understand the frequency of the data points, and the overall time span of the dataset. We also want to check for any missing timestamps that could affect our analysis - this is different from tabular data where we just check for missing values in the data columns themselves. Here, we know that if a timestamp is missing, it means we have no data for that time point.

In [4]:
def describe_time_index(ts: pd.Series) -> dict:
    """
    Describe the time index of a DataFrame in terms of frequency, time span, and missing timestamps.

    Parameters:
    ts (pd.Series): Series with a datetime index.
    
    Returns:
    dict: A dictionary containing the inferred frequency, start time, end time, and missing timestamps
    of the time index.

    Raises:
    ValueError: If the DataFrame index is not a datetime type.
    """

    index = ts.index
    if not pd.api.types.is_datetime64_any_dtype(index):
        raise ValueError("DataFrame index must be a datetime type.")

    # Frequency
    inferred_freq = pd.infer_freq(index)
    
    # Time span
    start_time = index.min()
    end_time = index.max()
    
    # Missing timestamps
    full_time_index = pd.date_range(start=start_time, end=end_time, freq=inferred_freq)
    missing_timestamps = full_time_index.difference(index)
    
    return {
        'inferred_frequency': inferred_freq,
        'start_time': start_time,
        'end_time': end_time,
        'n_missing_timestamps': len(missing_timestamps),
        'missing_timestamps': missing_timestamps.tolist()
    }

In [5]:
describe_time_index(df)

{'inferred_frequency': 'h',
 'start_time': Timestamp('2015-01-01 00:00:00+0000', tz='UTC'),
 'end_time': Timestamp('2020-09-30 23:00:00+0000', tz='UTC'),
 'n_missing_timestamps': 0,
 'missing_timestamps': []}

We can see that this data has hourly frequency with no missing timestamps. This removes concerns about irregular sampling or gaps in the data that could complicate our analysis later on.

The next step is to visualize the time series to get a sense of its overall structure and any obvious patterns or anomalies. This is the most intuitive step in time series EDA, as visual inspection can often reveal trends, seasonality, and outliers that statistical tests might miss.

In [6]:
def plot_time_series(ts: pd.Series) -> go.Figure:
    """
    Plot a univariate time series DataFrame using Plotly.

    Parameters:
    df (pd.DataFrame): DataFrame with a datetime index and a single column representing the time series data.

    Returns:
    fig: Plotly figure object displaying the time series.
    """

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ts.index, y=ts, mode='lines', name=ts.name))
    fig.update_layout(title=ts.name + ' time series',
                    xaxis_title='Time')
    fig.update_layout(template='plotly_white')

    return fig

In [7]:
fig = plot_time_series(df['load'])
fig.show()

# Measures of Distribution

The next step in our time series exploratory data analysis is to examine the distribution of the data values themselves. Understanding the distribution helps us identify characteristics such as skewness, kurtosis, and the presence of outliers, which can inform our choice of modeling techniques later on. 

We make two histograms: one for the full dataset, and another for the differenced data to see how changes from one point in time to another are distributed. This can be very useful for identifying trends in the dataset. We also make quick calculations of key statistics like mean, median, standard deviation, skewness, and kurtosis to summarize the distribution.

Per *Practical Time Series Analysis*, p95:
> In a time series context, a (histogram) of the difference of the data is often more interest‐ ing than a (histogram)) of the untransformed data. After all, in a time series context, often (and particularly in finance) what is most interesting is how a value changes from one measurement to the next rather than the value’s actual measurement. This is particularly true for plotting, because taking the histogram of data with a trend in it does not produce a very informative visualization.

In [8]:
def describe_distribution(ts: pd.Series) -> dict:
    """
    Describe the distribution of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.

    Returns:
    dict: A dictionary containing key statistics of the distribution.
    """

    return {

        'min': ts.min(),
        '25th_percentile': ts.quantile(0.25),
        '50th_percentile': ts.quantile(0.50),
        '75th_percentile': ts.quantile(0.75),
        'max': ts.max(),
        'range': ts.max() - ts.min(),
        'mean': ts.mean(),
        'std_dev': ts.std(),
        'skewness': ts.skew(),
        'kurtosis': ts.kurtosis()
    }

def plot_distribution_histogram(ts: pd.Series, orientation: str = 'v') -> go.Figure:
    """
    Plot a histogram of the data.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.

    Returns:
    fig: Plotly figure object displaying the histogram.
    """

    fig = go.Figure()
    if orientation == 'h':
        fig.add_trace(go.Histogram(y=ts, nbinsy=50))
        fig.update_layout(title='Distribution of ' + ts.name,
                          yaxis_title=ts.name,
                          xaxis_title='Count')
    elif orientation == 'v':
        fig.add_trace(go.Histogram(x=ts, nbinsx=50))
        fig.update_layout(title='Distribution of ' + ts.name,
                        xaxis_title=ts.name,
                        yaxis_title='Count')
    fig.update_layout(template='plotly_white')

    return fig

In [9]:
# regular histogram
distribution_stats = describe_distribution(df['load'])
distribution_histogram = plot_distribution_histogram(df['load'])

print(distribution_stats)
distribution_histogram.show()

{'min': np.float64(664.0), '25th_percentile': np.float64(5941.0), '50th_percentile': np.float64(7032.0), '75th_percentile': np.float64(8079.0), 'max': np.float64(10803.0), 'range': np.float64(10139.0), 'mean': np.float64(7070.148630952381), 'std_dev': np.float64(1400.2631669359246), 'skewness': np.float64(0.14004809936323936), 'kurtosis': np.float64(-0.5671520761708422)}


In [10]:
# differenced histogram and stats
diff_ts = df['load'].diff().dropna()
diff_distribution_stats = describe_distribution(diff_ts)
diff_distribution_histogram = plot_distribution_histogram(diff_ts)

print(diff_distribution_stats)
diff_distribution_histogram.show()

{'min': np.float64(-4451.0), '25th_percentile': np.float64(-227.0), '50th_percentile': np.float64(-63.0), '75th_percentile': np.float64(152.0), 'max': np.float64(5283.0), 'range': np.float64(9734.0), 'mean': np.float64(-0.010932756602313538), 'std_dev': np.float64(364.92299580636114), 'skewness': np.float64(0.9673402611826681), 'kurtosis': np.float64(3.495324363795505)}


# Stationarity

Stationarity can be a slippery concept to grasp, but it is crucial for many time series modeling techniques. A stationary time series has statistical properties, such as mean and variance, that do not change over time and allow us to expect that the system's long-term past behavior will continue to reflect in its future behavior. Many models, such as ARIMA, assume stationarity, so it's important to assess this property in our data before selecting a modeling approach.

The mathematical definition of stationarity can be complex, but for practical purposes, we can use statistical tests to assess whether our time series is stationary. One common test is the **Augmented Dickey-Fuller (ADF)** test, which tests the null hypothesis that a unit root is present in the time series. If we can reject this null hypothesis, we can conclude that the time series is stationary.

In [11]:
from statsmodels.tsa.stattools import adfuller

def describe_stationarity(ts: pd.Series) -> dict:
    """
    Perform the Augmented Dickey-Fuller test to assess the stationarity of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.

    Returns:
    dict: A dictionary containing the ADF test results.
    """

    adf_result = adfuller(ts)
    adf_dict ={
        'adf_statistic': adf_result[0],
        'p_value': adf_result[1],
        'used_lag': adf_result[2],
        'n_obs': adf_result[3],
        'critical_values': adf_result[4],
        'ic_best': adf_result[5]
    }

    if adf_dict['p_value'] < adf_dict['critical_values']['5%']:
        adf_dict['stationarity'] = True
        return adf_dict
    else:
        adf_dict['stationarity'] = False
        return adf_dict

    

In [12]:
describe_stationarity(df['load'])

{'adf_statistic': np.float64(-16.67800764176947),
 'p_value': np.float64(1.524895598250865e-29),
 'used_lag': 57,
 'n_obs': 50342,
 'critical_values': {'1%': np.float64(-3.4304799041251957),
  '5%': np.float64(-2.8615974149640566),
  '10%': np.float64(-2.5668005600849857)},
 'ic_best': np.float64(648211.7023811574),
 'stationarity': False}

For this time series, the ADF test indicates that we can reject the null hypothesis, suggesting that the time series is stationary. This is a good sign for modeling, as many time series models require stationarity. 


# Self-Correlation

Next, let's examine the self-correlation of the time series. Per wikipedia:

> Self-correlation measures the correlation of a signal with a delayed copy of itself. Essentially, it quantifies the similarity between observations of a random variable at different points in time. 

There are two types od self-correlation that are commonly used in time series analysis: **autocorrelation** and **partial autocorrelation**. In both, we calculate the correlation between the time series and lagged versions of itself and see if they fall outside of a critical value range, indicating significant correlation at that lag.

### Autocorrelation and Autocorrelation Function (ACF)

 The first type of self-correlation we will look at is the **autocorrelation** and the **autocorrelation function (ACF)**, which measures the correlation between the time series and its lagged values. The ACF helps us identify patterns such as seasonality and trends in the data by showing how past values influence current values over different time lags.

In [13]:
from statsmodels.tsa.stattools import acf

def describe_acf(ts: pd.Series, nlags: int = 24) -> dict:
    """
    Describe the autocorrelation function (ACF) of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.
    nlags (int): Number of lags to compute the ACF for.

    Returns:
    dict: A dictionary containing the ACF values for each lag.
    """

    acf_values = acf(ts, nlags=nlags)
    acf_dict = {f'lag_{i}': acf_values[i] for i in range(len(acf_values))}
    return acf_dict

def plot_acf(ts: pd.Series, nlags: int = 24) -> go.Figure:
    """
    Plot the autocorrelation function (ACF) of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.
    nlags (int): Number of lags to include in the ACF plot.

    Returns:
    fig: Plotly figure object displaying the ACF.
    """

    acf_values = acf(ts, nlags=nlags)
    
    critical_value = 1.96 / (len(ts) ** 0.5)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=list(range(len(acf_values))), y=acf_values))
    fig.add_shape(type='line',
                    x0=0, y0=critical_value, x1=nlags, y1=critical_value,
                    line=dict(color='Red', dash='dash'))
    fig.add_shape(type='line',
                    x0=0, y0=-critical_value, x1=nlags, y1=-critical_value,
                    line=dict(color='Red', dash='dash'))
    
    fig.update_layout(title='Autocorrelation Function (ACF)',
                      xaxis_title='Lags',
                      yaxis_title='ACF')
    fig.update_layout(template='plotly_white')

    return fig

In [14]:
plot_acf(df['load'], nlags=168).show()

This has some very expected but also interesting results. This is hourly data and we have run autocorrelation out to 168 lags, which is one week. We can see correlation peaks at lags of 24, 48, 72, etc., indicating a strong daily seasonality in the data. There are also stronger corrleation peaks at lags of 168, indicating a weekly seasonality as a result of different energy usage patterns on weekends versus weekdays.

Let's resample the data to daily frequency and re-examine the ACF to see how the self-correlation structure changes with a different time granularity a correspondingly longer lag period (365 days). 

In [15]:
rs_load = df['load'].resample('D').sum()

plot_acf(rs_load, nlags=365).show()

As expected since "the ACF of a periodic function has the same periodicity as the original process", the daily resampling smooths out the high-frequency variations seen in the hourly data, leading to a different autocorrelation structure. The ACF now shows significant correlations at lags corresponding to weekly and annual cycles, reflecting broader seasonal patterns in energy demand. 

### Partial Autocorrelation and Partial Autocorrelation Function (PACF)

The partial autocorrelation function (PACF) measures the correlation between the time series and its lagged values, controlling for the values of the time series at all shorter lags. In other words, it shows the direct effect of a lagged value on the current value, removing the influence of intermediate lags. It does this by calculating the correlation between the time series and its lagged values after removing the effects of the intervening lags through a series of linear regressions.

To determine the partial correlation of a single lag at k, the PACF performs the following steps:
1. Regress the time series on all lags from 1 to k-1 and obtain the residuals.
2. Regress the time series lagged by k on all lags from 1 to k-1 and obtain the residuals.
3. Calculate the correlation between the two sets of residuals obtained in steps 1 and 2. This correlation is the partial autocorrelation at lag k.

This can be very useful for identifying the appropriate lag order for autoregressive models, as it helps to isolate the effect of each lag without the confounding influence of other lags.

In [16]:
from statsmodels.tsa.stattools import pacf

def describe_pacf(ts: pd.Series, nlags: int = 24) -> dict:
    """
    Describe the partial autocorrelation function (PACF) of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.
    nlags (int): Number of lags to compute the PACF for.

    Returns:
    dict: A dictionary containing the PACF values for each lag.
    """

    pacf_values = pacf(ts, nlags=nlags)
    pacf_dict = {f'lag_{i}': pacf_values[i] for i in range(len(pacf_values))}
    return pacf_dict

def plot_pacf(ts: pd.Series, nlags: int = 24) -> go.Figure:
    """
    Plot the partial autocorrelation function (PACF) of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.
    nlags (int): Number of lags to include in the PACF plot.

    Returns:
    fig: Plotly figure object displaying the PACF.
    """

    pacf_values = pacf(ts, nlags=nlags)

    critical_value = 1.96 / (len(ts) ** 0.5)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=list(range(len(pacf_values))), y=pacf_values))
    fig.add_shape(type='line',
                    x0=0, y0=critical_value, x1=nlags, y1=critical_value,
                    line=dict(color='Red', dash='dash'))
    fig.add_shape(type='line',
                    x0=0, y0=-critical_value, x1=nlags, y1=-critical_value,
                    line=dict(color='Red', dash='dash'))
    fig.update_layout(title='Partial Autocorrelation Function (PACF)',
                      xaxis_title='Lags',
                      yaxis_title='PACF')
    fig.update_layout(template='plotly_white')

    return fig

In [17]:
plot_pacf(df['load']).show()

From *Practical Time Series Analysis*, p95:
> The PACF reveals which correlations are “true” informative correlations for specific lags rather than redundancies. This is invaluable for knowing when we have collected enough information to get a sufficiently long window at a proper temporal scale for our data.

We can see that the PACF for the hourly data shows the most significant partial correlations at lags of 1, 2, 3 indicating that these lags have a direct influence on the current value of the time series.

In [18]:
rs_load = df['load'].resample('D').sum()

plot_pacf(rs_load, nlags=365).show()

# Putting it All Together: An EDA Class

This should be enough to get us started with time series exploratory data analysis! We have covered data loading and preprocessing, understanding the time index, examining measures of distribution, assessing stationarity, and analyzing self-correlation through ACF and PACF.

In [19]:
%load_ext autoreload


In [20]:
%autoreload 2

import sys
from pathlib import Path

# Add the "src" directory to Python path
sys.path.append(str(Path.cwd().parent / "src"))

from ts_trove.eda.univaritate_eda import UnivariateEDA
from ts_trove.eda.univariate_eda_report import UnivaritateEDAReport

In [21]:
uv_eda = UnivariateEDA(ts=df['load'])
report = UnivaritateEDAReport(univariate_eda=uv_eda)
report.generate(output_path=Path('../reports/univariate_eda_report.html'))